In [2]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
import math




# Import Sheet as csv 

In [3]:
ps = pd.read_csv('patch_seq_log_mouse.csv')




# Format sheet 

In [4]:
ps = ps[["Date", "User", "File", "Post patch?", "Notes", "Conc.", ">400", "Ratio", "SM_QC_PF", "Bad dates"]]
ps.dropna(subset = ["SM_QC_PF", "Post patch?"], inplace = True)
ps["User"] = ps["User"].astype("category")
ps["SM_QC_PF"] = ps["SM_QC_PF"].astype("category")
ps["Post patch?"] = ps["Post patch?"].astype("category")
ps["Conc."] = pd.to_numeric(ps["Conc."].str.replace(' ',''), errors ='force')
ps[">400"] = pd.to_numeric(ps[">400"].str.replace(' ',''), errors ='force')
ps["Date"] = pd.to_datetime(ps["Date"], format = '%y%m%d')
ps = ps[ps["SM_QC_PF"] != "cnt"]
ps = ps[ps["Bad dates"] != "x"]
ps.head()

,Date,User,File,Post patch?,Notes,Conc.,>400,Ratio,SM_QC_PF,Bad dates
251,2016-03-18,P5,Ndnf - brl_160318_01,nr,NaN,106.83,NaN,NaN,fail,NaN
252,2016-03-18,P5,Ndnf - brl_160318_02,nr,Cell died at 6 mins,3649.03,NaN,NaN,fail,NaN
253,2016-03-18,P5,Ndnf - brl_160318_03,nr,NaN,70.89,NaN,NaN,fail,NaN
254,2016-03-18,P5,Ndnf - brl_160318_04,Outside-Out,"broke in on its own, leaky",320.83,278.0,0.81,pass,NaN
255,2016-03-18,P5,Ndnf - brl_160318_05,nr,NaN,425.15,NaN,NaN,fail,NaN


In [5]:
ps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3622 entries, 251 to 7426
Data columns (total 10 columns):
Date           3622 non-null datetime64[ns]
User           3614 non-null category
File           3621 non-null object
Post patch?    3622 non-null category
Notes          2487 non-null object
Conc.          3507 non-null float64
>400           3311 non-null float64
Ratio          3491 non-null object
SM_QC_PF       3622 non-null category
Bad dates      127 non-null object
dtypes: category(3), datetime64[ns](1), float64(2), object(4)
memory usage: 237.2+ KB


# Percentage of nucleated patches passing SM_QC 

In [6]:
mask = ps["Post patch?"] == "Nucleated"
mask
nuc = ps[mask]
nuc.head()
nuc_count = nuc["Post patch?"].count()
nuc_count

1055

In [7]:
ps["Post patch?"].value_counts()

Nucleated             1055
No-Seal                969
Outside-Out            907
Partial-Nucleus        411
nr                     178
Entire-Cell             38
nucleus_present         32
nucleus_absent          14
no-seal                  5
no_nucleus_visible       4
nucleus_visible          3
Outside-Out?             1
outside-out              1
entire_cell              1
missing                  1
nucelated                1
no -seal                 1
Name: Post patch?, dtype: int64

In [8]:
pas = ps["SM_QC_PF"] == "pass"
pas.head()

251    False
252    False
253    False
254     True
255    False
Name: SM_QC_PF, dtype: bool

In [9]:
np = ps[mask & pas]
np

,Date,User,File,Post patch?,Notes,Conc.,>400,Ratio,SM_QC_PF,Bad dates
268,2016-04-06,P5,Ndnf - brl_160406_04,Nucleated,NaN,209.76,167.0,0.69,pass,NaN
280,2016-04-07,P1,Ndnf - kjh_160407_03,Nucleated,"cell swelled, no fluorescence visible in pipette",114.09,96.0,0.73,pass,NaN
303,2016-04-11,P5,Rorb - brl_160411_02,Nucleated,NaN,125.35,92.0,0.63,pass,NaN
320,2016-04-12,P5,Rorb - brl_160412_01,Nucleated,"couldnt see flourescence - cell too deep, but ...",195.16,183.0,0.87,pass,no slide in LIMS
360,2016-04-18,P5,Chat - brl_160418_02,Nucleated,"coudlnt get gigaseal, put still saw flur in pi...",135.75,116.0,0.85,pass,NaN
363,2016-04-18,P5,Chat - brl_160418_05,Nucleated,lost seal at 3 mins but still able to pull ou...,158.45,113.0,0.62,pass,NaN
376,2016-04-19,P1,Chat - kjh_160419_03,Nucleated,saw fluorescence in pipette,423.02,203.0,0.48,pass,NaN
395,2016-04-20,P1,Vip - kjh_160420_10,Nucleated,"saw fluorescence in pipette, cell dimmed",470.63,338.0,0.64,pass,NaN
439,2016-04-26,P5,Nos1 - brl_160426_02,Nucleated,NaN,353.39,282.0,0.73,pass,cell is between 01 and 02 (different than cell...
441,2016-04-26,P5,Nos1 - brl_160426_04,Nucleated,NaN,425.15,332.0,0.72,pass,NaN


In [10]:
nuc_pass = np["Post patch?"].count()
nuc_pass
#np["Post patch?"].count()

896

In [11]:
float(nuc_pass)/nuc_count

0.8492890995260663

# Bubbles Analysis 

In [25]:
bubbles = ps.dropna(subset = ["Notes"])
bubbles

,Date,User,File,Post patch?,Notes,Conc.,>400,Ratio,SM_QC_PF,Bad dates
252,2016-03-18,P5,Ndnf - brl_160318_02,nr,Cell died at 6 mins,3649.03,NaN,NaN,fail,NaN
254,2016-03-18,P5,Ndnf - brl_160318_04,Outside-Out,"broke in on its own, leaky",320.83,278.00,0.81,pass,NaN
256,2016-03-18,P5,Ndnf - brl_160318_06,nr,"had isseus breaking in, applied a lot of press...",188.27,NaN,NaN,fail,NaN
258,2016-03-18,P5,Ndnf - brl_160318_x,nr,cell busted seal but still sucked up fluroescence,211.35,200.00,0.84,pass,NaN
259,2016-03-18,P5,Ndnf - brl_160318_08,nr,lost cell at 3mins,4.69,4.00,NaN,fail,NaN
267,2016-04-06,P5,Ndnf - brl_160406_03,Outside-Out,wiped with kim wipe without water before loadi...,100.61,37.65,NaN,fail,NaN
269,2016-04-06,P5,Ndnf - brl_160406_05,Partial-Nucleus,cell swelled,125.58,92.00,0.61,pass,NaN
270,2016-04-06,P5,Ndnf - brl_160406_06,nr,"fluorescence moved up into pipette, no longer ...",92.49,78.00,0.75,pass,NaN
279,2016-04-07,P1,Ndnf - kjh_160407_02,nr,no fluorescence visible in pipette,40.19,33.54,NaN,fail,NaN
280,2016-04-07,P1,Ndnf - kjh_160407_03,Nucleated,"cell swelled, no fluorescence visible in pipette",114.09,96.00,0.73,pass,NaN


In [13]:
def make_mask(v,x):
    mask = v["Notes"].str.contains(x)
    df = v[mask]
    return df

def pass_only(y):
    mask = y["SM_QC_PF"] == "pass"
    df2 = y[mask]
    return df2

#Should also define a function to calculate pass percentages

In [14]:
def get_count(x):    
    count = x[">400"].count()
    return count

def get_sqrtct(x):
    sqrt_ct = math.sqrt(get_count(x))
    return sqrt_ct

def calc_mean(x):
    mean = x[">400"].mean()
    return mean

def calc_std(x):
    std_dev = x[">400"].std()
    return std_dev

def calc_stderr(x):
    std_err = calc_std(x) / get_sqrtct(x)
    return std_err

def calc_pp(passing,total):
    pp = float(get_count(passing)) / get_count(total)
    return pp

# No Bubbles Stats - All Samples

In [15]:
nb = make_mask(bubbles,"No Bubbles")

nb_mean = calc_mean(nb)
print "mean", nb_mean

nb_stderr = calc_stderr(nb)
print "standard error of the mean", nb_stderr

mean 1366.6611236
standard error of the mean 47.6815409343


# No Bubbles Stats - SMQC Pass Only 

In [16]:
nb_p = pass_only(make_mask(bubbles,"No Bubbles"))

nb_mean = calc_mean(nb_p)
print "mean =", nb_mean

nb_stderr = calc_stderr(nb_p)
print "standard error of the mean =", nb_stderr

nb_pp = calc_pp(nb_p,nb)
print "pass percentage =", nb_pp

mean = 1578.02451791
standard error of the mean = 50.7005797402
pass percentage = 0.815730337079


# Small Bubbles Stats - All Samples 

In [17]:
sb = make_mask(bubbles,"Small Bubbles")

sb_mean = calc_mean(sb)
print "mean", sb_mean

sb_stderr = calc_stderr(sb)
print "standard error of the mean", sb_stderr

mean 1369.29014085
standard error of the mean 86.5840364432


# Small Bubbles Stats - SMQC Pass Only 

In [18]:
sb_p = pass_only(make_mask(bubbles,"Small Bubbles"))

sb_mean = calc_mean(sb_p)
print "mean", sb_mean

sb_stderr = calc_stderr(sb_p)
print "standard error of the mean", sb_stderr

sb_pp = calc_pp(sb_p,sb)
print "pass percentage =", sb_pp

mean 1672.03350785
standard error of the mean 107.5958286
pass percentage = 0.76861167002


# Medium bubbles Stats - All Samples 

In [19]:
mb = make_mask(bubbles,"Medium Bubbles")

mb_mean = calc_mean(mb)
print "mean =", mb_mean

mb_stderr = calc_stderr(mb)
print "standard error of the mean =", mb_stderr

mean = 1334.71764706
standard error of the mean = 106.315505511


# Medium Bubbles Stats - SMQC Pass Only

In [20]:
mb_p = pass_only(make_mask(bubbles, "Medium Bubbles"))

mb_mean = calc_mean(mb_p)
print "mean =", mb_mean

mb_stderr = calc_stderr(mb_p)
print "standard error of the mean =", calc_stderr(mb_p)

mb_pp = calc_pp(mb_p,mb)
print "pass percentage =", mb_pp

print get_count(mb)
print get_count(mb_p)

mean = 1594.10487805
standard error of the mean = 115.048434532
pass percentage = 0.803921568627
102
82


# Large Bubbles Stats - All Samples 

In [21]:
lb = make_mask(bubbles,"Large Bubbles")

lb_mean = calc_mean(lb)
print "mean =", lb_mean

lb_stderr = calc_stderr(lb)
print "standard error of the mean", lb_stderr

mean = 1349.54736842
standard error of the mean 188.811785898


# Large Bubbles Stats - SMQC Pass Only 

In [22]:
lb_p = pass_only(make_mask(bubbles,"Large Bubbles"))

lb_mean = calc_mean(lb_p)
print "mean =", lb_mean

lb_stderr = calc_stderr(lb_p)
print "standard error of the mean =", lb_stderr

lb_pp = calc_pp(lb_p,lb)
print "pass percentage =", lb_pp

mean = 1653.30333333
standard error of the mean = 205.483099872
pass percentage = 0.789473684211


# Solution up Pipette - All Samples 

In [23]:
sp = make_mask(bubbles,"Solution in Pipette Shank")

sp_mean = calc_mean(sp)
print "mean =", sp_mean

sp_stderr = calc_stderr(sp)
print "standard error of the mean =", sp_stderr

mean = 914.647368421
standard error of the mean = 295.180430139


# Solution up Pipette - SMQC Pass Only 

In [24]:
sp_p = pass_only(make_mask(bubbles, "Solution in Pipette Shank"))

sp_mean = calc_mean(sp_p)
print "mean =", sp_mean

sp_stderr = calc_stderr(sp_p)
print "standard error of the mean =", sp_stderr

sp_pp = calc_pp(sp_p,sp)
print "pass percentage =", sp_pp

print get_count(sp)
print get_count(sp_p)

mean = 1410.75454545
standard error of the mean = 458.825817884
pass percentage = 0.578947368421
19
11
